In [1]:
import pysam
import kbbq.bloom
import kbbq.read
import khmer
import numpy as np
import scipy.stats as ss
import collections

In [2]:
ksize = 32
memstr = '2G'
maxmem = khmer.khmer_args.memory_setting(memstr)
readsfile = 'tests/data/reads.fq'

def create_countgraph():
    fake_args = collections.namedtuple('fake_args', ['max_memory_usage', 'n_tables'])
    fake = fake_args(max_memory_usage = maxmem, n_tables = 4)
    tablesize = khmer.khmer_args.calculate_graphsize(fake, 'countgraph', multiplier = 1.0)
    countgraph = khmer.Countgraph(ksize, tablesize, fake.n_tables)
    return countgraph

In [3]:
countgraph = create_countgraph()
countgraph.consume_seqfile(readsfile)
count_fpr = khmer.calc_expected_collisions(countgraph, force = False, max_false_pos = .15)
print(count_fpr)

5.884812846252029e-12


In [5]:
nodegraph = kbbq.bloom.create_empty_nodegraph(ksize = ksize, max_mem = memstr)
node_fpr = khmer.calc_expected_collisions(countgraph, force = False, max_false_pos = .15)
print(node_fpr)

5.884812846252029e-12


In [ ]:
def count_from_file(filename, graph, sampling_rate):
    for read in pysam.FastxFile(filename, 'r'):
        r = kbbq.read.ReadData.from_fastq(r)
        kbbq.bloom.count_read(r, nodegraph, sampling_rate)

In [ ]:
count_from_file(readsfile, nodegraph, sampling_rate)